In [1]:
import pandas as pd
import matplotlib as plt
import numpy as np

## Function for Reading Files

In [2]:
def get_data(year):
    new_filename = f'{year}.txt'
    data: pd.DataFrame = pd.read_csv(f'datasets/babynames/yob{new_filename}', names = ['Name', 'Sex', 'BirthCount'])

    data['Year'] = year
    return data

## Joining Files Together

In [6]:
min_year = 1880
max_year = 2011

names = pd.DataFrame([])

for year in range(min_year, max_year):
    frame = get_data(year)
    names = pd.concat([names, frame], ignore_index = True)

In [7]:
names

,Name,Sex,BirthCount,Year
0,Mary,F,7065,1880
1,Anna,F,2604,1880
2,Emma,F,2003,1880
3,Elizabeth,F,1939,1880
4,Minnie,F,1746,1880
...,...,...,...,...
1690779,Zymaire,M,5,2010
1690780,Zyonne,M,5,2010
1690781,Zyquarius,M,5,2010
1690782,Zyran,M,5,2010


In [ ]:
names[names['Name'] == 'Mary']

,Name,Sex,BirthCount,Year
1513354,Zyquarius,M,9,2005
1587335,Zyquarius,M,5,2007
1622340,Zyquarius,M,5,2008
1656941,Zyquarius,M,5,2009
1690781,Zyquarius,M,5,2010


## Grouping Data By Feature

In [ ]:
group_s = names.groupby('Sex', as_index = False)

In [ ]:
print(f'Dataset Contains {len(group_s.size())} unique Sex groups: \n\n', group_s.size())

## Plotting Births Over Time

In [ ]:
total_births = names.pivot_table('BirthCount', index='Year', columns='Sex', aggfunc=sum)

In [ ]:
total_births.plot(title = 'Total Births by Sex and Year')

## Most Common Names

In [ ]:
def add_proportion(group):
    group['Proportion'] = group['BirthCount'] / group['BirthCount'].sum()
    return group

In [ ]:
g = names.groupby(['Year', 'Sex'], group_keys = False)
g.size()

In [ ]:
df = g.apply(add_proportion)
df.iloc[df[df['Sex'] == 'M']['Proportion'].idxmax()]

## Sorting Values

In [ ]:
def get_top(group, results = 1000):
    if results > 0 and results <= len(group):
        return group.sort_values('BirthCount', ascending = False)[:results]

In [ ]:
grouped = names.groupby(['Year', 'Sex'])
top = grouped.apply(get_top)

In [ ]:
top

In [ ]:
# drop the year grouping
top = top.reset_index(drop=True)
top

## Shuffle and Sampling

In [ ]:
shuffle_index = np.random.permutation(top.index)
shuffle_index

In [ ]:
shuffled_df = top.reindex(shuffle_index)
sample = shuffled_df.head(1000)

In [ ]:
sample

In [ ]:
males = sample[sample['Sex'] == 'M']
males

In [ ]:
females = sample[sample['Sex'] == 'F']
females

In [ ]:
total_births = top.pivot_table('BirthCount', index='Year', columns='Name', aggfunc=sum)

total_births

In [ ]:
subset = total_births[['John', 'Harry', 'Mary', 'Marylin']]
subset.plot(subplots = True, figsize = (12,10), ylabel='BirthCount', xlabel='Year')

## Unisex Names Over Time

In [ ]:
all_names = pd.Series(top['Name'].unique())
all_names

In [ ]:
lesley_like = all_names[all_names.str.contains('Lesl')]
lesley_like

In [ ]:
filtered = top[top["Name"].isin(lesley_like)]
filtered

In [ ]:
filtered.groupby('Name')['BirthCount'].sum()

In [ ]:
table = filtered.pivot_table('BirthCount', index = 'Year', columns = 'Sex', aggfunc = 'sum')

table

In [ ]:
table = table.div(table.sum(axis='columns'), axis='index')

table

In [ ]:
table.plot(style={'M': 'k-', 'F': 'k--'})